In [49]:
# !pip install "google-cloud-aiplatform>=1.38" vertexai
#TODO: gcloud auth application-default login to authenticate

In [50]:
import vertexai
from vertexai.generative_models import GenerativeModel

# TODO(developer): Update and un-comment below line
project_id = "gvt0031-gcp-152-govtext-ds"

vertexai.init(project=project_id, location="asia-southeast1")

model = GenerativeModel(model_name="gemini-1.5-flash-001")

## Preprocess input

In [51]:
llama_predictions_path = "/home/watson_chua/efs/axolotl/data/predictions/hansard/llama3_8b_lora.jsonl"
gemma_predictions_path = "/home/watson_chua/efs/axolotl/data/predictions/hansard/gemma2_9b_lora.jsonl"
# gpt4_predictions_path = "/home/watson_chua/efs/axolotl/data/input_data/data/gpt4_answers_by_hy_doc.jsonl"
gpt4_predictions_path = "/home/watson_chua/efs/axolotl/data/input_data/data/gpt4_answers_by_hy_doc_concise.jsonl"

In [52]:
import json
with open(llama_predictions_path, 'r') as f:
    llama_lines = f.readlines()

with open(gemma_predictions_path, 'r') as f:
    gemma_lines = f.readlines()

with open(gpt4_predictions_path, 'r') as f:
    gpt4_lines = f.readlines()

llama_lines = [json.loads(l) for l in llama_lines]
gemma_lines = [json.loads(l) for l in gemma_lines]
gpt4_lines = [json.loads(l) for l in gpt4_lines]

In [53]:
len(llama_lines), len(gemma_lines), len(gpt4_lines)

(304, 304, 304)

In [54]:
llama_lines[0]

{'generation': 'As at end-January 2024, there were 101,464 persons who did not vote in the last Presidential election but have since applied to restore their names to the Registers of Electors. All such applications have been approved after being verified by the Elections Department.',
 'input': "<|start_header_id|>system<|end_header_id|>You are a public servant. Your task is to reply to a parliamentary question given a list of supporting points.<|eot_id|><|start_header_id|>user<|end_header_id|>\nQuestion:Mr Leong Mun Wai asked the Prime Minister (a) since the conclusion of the 2023 Presidential Election, how many non-voters have applied to restore their names to the Registers of Electors; and (b) how many of these applications are successful.\n\nSupporting points: **Title: Applications to Restore Names to Registers of Electors**\n\n**Executive Summary:**\n\nAs of the end of January 2024, a total of 101,464 non-voters from the 2023 Presidential Election have successfully applied to res

In [55]:
gpt4_lines[0]['question']

'Mr Leong Mun Wai asked the Prime Minister (a) since the conclusion of the 2023 Presidential Election, how many non-voters have applied to restore their names to the Registers of Electors; and (b) how many of these applications are successful.'

In [56]:
llama_lines[0]['input'].split("<|end_header_id|>\nQuestion:", maxsplit=1)[-1].split("\n\nSupporting points:")[0]

'Mr Leong Mun Wai asked the Prime Minister (a) since the conclusion of the 2023 Presidential Election, how many non-voters have applied to restore their names to the Registers of Electors; and (b) how many of these applications are successful.'

In [57]:
llama_questions = [q['input'].split("<|end_header_id|>\nQuestion:", maxsplit=1)[-1].split("\n\nSupporting points:")[0] for q in llama_lines]

In [58]:
for i,l in enumerate(gpt4_lines):
    if l['question'] not in llama_questions:
        print(i, l['question'])

In [59]:
import pandas as pd 
df_gpt4 = pd.DataFrame(gpt4_lines)

In [60]:
df_duplicated = df_gpt4[df_gpt4['question'].duplicated()]

In [61]:
df_gpt4_deduplicated = df_gpt4.drop_duplicates('question')

In [62]:
df_gpt4_deduplicated.head()

,status,reference_answer,summary_points,points,title,subtitle,question_speaker,answer_speaker,question,answer,filename,hypothetical_document,date,gpt4_answer_by_hy_doc
0,answered,,"[101,464 non-voters from the 2023 Presidential...","101,464 non-voters from the 2023 Presidential ...",Applications to Restore Names to Registers of ...,None,Mr Leong Mun Wai,Mr Chan Chun Sing (for the Prime Minister),Mr Leong Mun Wai asked the Prime Minister (a) ...,Mr Chan Chun Sing (for the Prime Minister): As...,sitting_2024-02-29,**Title: Applications to Restore Names to Regi...,2024-02-29 00:00:00,Title: Applications to Restore Names to Regist...
1,answered,,"[From 2021 to 2023, over 23,300 companies obta...","From 2021 to 2023, over 23,300 companies obtai...",Applications under Enterprise Financing Scheme,None,Mr Liang Eng Hwa,Mr Gan Kim Yong,Mr Liang Eng Hwa asked the Minister for Trade ...,"Mr Gan Kim Yong: From 2021 to 2023, over 23,30...",sitting_2024-02-29,**Title: Applications under Enterprise Financi...,2024-02-29 00:00:00,### Applications under Enterprise Financing Sc...
2,answered,,"[About 36,400 companies have received support ...","About 36,400 companies have received support u...",Applications for Productivity Solutions Grant ...,None,Mr Liang Eng Hwa,Mr Gan Kim Yong,Mr Liang Eng Hwa asked the Minister for Trade ...,"Mr Gan Kim Yong: In the last three years, abou...",sitting_2024-02-29,### Applications for Productivity Solutions Gr...,2024-02-29 00:00:00,Title: Applications for Productivity Solutions...
3,answered,,[Schools have autonomy to decide the end times...,Schools have autonomy to decide the end times ...,Time Guidelines for Schools to Cease Co-curric...,None,Mr Leong Mun Wai,Mr Chan Chun Sing,Mr Leong Mun Wai asked the Minister for Educat...,Mr Chan Chun Sing: Schools have the autonomy t...,sitting_2024-02-29,**Title: Time Guidelines for Schools to Cease ...,2024-02-29 00:00:00,Title: Time Guidelines for Schools to Cease Co...
4,answered,,[Rail operators have improved track design and...,Rail operators have improved track design and ...,Mitigation Measures to Reduce Noise from Railw...,None,Mr Leong Mun Wai,Mr Chee Hong Tat,Mr Leong Mun Wai asked the Minister for Transp...,"Over the years, rail operators have improved t...",sitting_2024-02-29,**Report: Mitigation Measures to Reduce Noise ...,2024-02-29 00:00:00,Title: Mitigation Measures to Reduce Noise fro...


In [63]:
for i, (index, row) in enumerate(df_gpt4_deduplicated.iterrows()):
    if row['question'] != llama_questions[i]:
        print(row['question'])

## Evaluation

In [165]:
prompt_three_way_template = """Given the following question, context, and three different answers a), b) and c), assess the answer based on the following criteria: 
 1) factual correctness according to the context
 2) similarity to model answer
 3) conciseness
 
Respond in JSON whether answer a), b), or c) is the better answer and state your reason using the following schema:

{{"winner": a, b, or c,
"reason": reason it is the better answer}}

Here is the information,

{question}

Context: {context}

Model Answer: {ground_truth}

Answer A: {answer_a}

Answer B: {answer_b}

Answer C: {answer_c}



Respond with only the JSON reply and nothing else. Use double quotes for the json schema and DO NOT use any double quotes in the values.
"""


prompt_two_way_template = """Given the following question, context, and two different answers a) and b), assess the answer based on the following criteria: 
 1) factual correctness according to the context
 2) similarity to model answer
 3) conciseness
 
Respond in JSON whether answer a) or b) is the better answer and state your reason using the following schema:

{{"winner": a or b,
"reason": reason it is the better answer}}

Here is the information,

{question}

Context: {context}

Model Answer: {ground_truth}

Answer A: {answer_a}

Answer B: {answer_b}

Respond with only the JSON reply and nothing else. Use double quotes for the json schema and DO NOT use any double quotes in the values.
"""

In [166]:
from tqdm.auto import tqdm
three_way_evaluation_results = []
gpt4_llama3_evaluation_results = []
gpt4_gemma2_evaluation_results = []
llama3_gemma2_evaluation_results = []


df_eval = df_gpt4_deduplicated
for i, (index, row) in tqdm(enumerate(df_eval.iterrows()), total=len(df_eval)):
    gpt_4_answer = row['gpt4_answer_by_hy_doc']
    ground_truth = row['answer']
    question = row['question']
    hypothetical_doc = row['hypothetical_document']
    llama3_answer = llama_lines[i]['generation']
    gemma2_answer = gemma_lines[i]['generation']


    prompt_three_way = prompt_three_way_template.format(context=hypothetical_doc, ground_truth=ground_truth, question=question, answer_a=gpt_4_answer, answer_b=llama3_answer, answer_c=gemma2_answer)
    prompt_gpt4_llama3 = prompt_two_way_template.format(context=hypothetical_doc, ground_truth=ground_truth, question=question, answer_a=gpt_4_answer, answer_b=llama3_answer)
    prompt_gpt4_gemma2 = prompt_two_way_template.format(context=hypothetical_doc, ground_truth=ground_truth, question=question, answer_a=gpt_4_answer, answer_b=gemma2_answer)
    prompt_llama3_gemma2 = prompt_two_way_template.format(context=hypothetical_doc, ground_truth=ground_truth, question=question, answer_a=llama3_answer, answer_b=gemma2_answer)


    try:
        response_three_way = model.generate_content(prompt_three_way).text
        response_gpt4_llama3 = model.generate_content(prompt_gpt4_llama3).text
        response_gpt4_gemma2 = model.generate_content(prompt_gpt4_gemma2).text
        response_llama3_gemma2 = model.generate_content(prompt_llama3_gemma2).text


    except ValueError as e:
        print(e)
        continue
    
    
    three_way_evaluation_results.append({'gemini_flash_evaluation': response_three_way, **row, 'llama3_answer': llama3_answer, 'gemma2_answer': gemma2_answer})
    gpt4_llama3_evaluation_results.append({'gemini_flash_evaluation': response_gpt4_llama3, **row, 'llama3_answer': llama3_answer})
    gpt4_gemma2_evaluation_results.append({'gemini_flash_evaluation': response_gpt4_gemma2, **row, 'gemma2_answer': gemma2_answer})
    llama3_gemma2_evaluation_results.append({'gemini_flash_evaluation': response_llama3_gemma2, **row, 'llama3_answer': llama3_answer, 'gemma2_answer': gemma2_answer})





100%|██████████| 304/304 [24:26<00:00,  4.82s/it]


In [167]:
output_root_dir = '/home/watson_chua/efs/axolotl/data/evaluations/hansard/'
subdir = 'gpt4_concise'

with open(output_root_dir + subdir + '/gpt4_llama3_gemma2_evaluation.jsonl', 'w') as f:
    for l in three_way_evaluation_results:
        f.write(json.dumps(l) + '\n')  

with open(output_root_dir + subdir + '/gpt4_llama3_evaluation.jsonl', 'w') as f:
    for l in gpt4_llama3_evaluation_results:
        f.write(json.dumps(l) + '\n')  

with open(output_root_dir + subdir + '/gpt4_gemma2_evaluation.jsonl', 'w') as f:
    for l in gpt4_gemma2_evaluation_results:
        f.write(json.dumps(l) + '\n')  

with open(output_root_dir + subdir + '/llama3_gemma2_evaluation.jsonl', 'w') as f:
    for l in llama3_gemma2_evaluation_results:
        f.write(json.dumps(l) + '\n')  

In [164]:
json.loads('\\\"hello\\\"')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [176]:
import re
def replace_inner_quotes(text):
    split_result  = re.split("\"\s*reason\s*\"", text, maxsplit=1)
    winner_str = split_result[0] 
    reason_str = split_result[1][split_result[1].index('"')+1: split_result[1].rfind('"')].replace('"','').replace("\\", "")
    return winner_str + "\"reason\": \"" + reason_str.strip() + '"}'

def clean_json_output(output):
    output = output.strip()
    if output.startswith("```json"):
        output = output[7:]
    if output.endswith("```"):
        output = output[:-3]
    cleaned_output = replace_inner_quotes(output.strip())
    # cleaned_output = output.strip()


    try:
        json_data = json.loads(cleaned_output)
    except json.JSONDecodeError as e:
        print(f"JSON decoding error: {e}")
        print(cleaned_output)
        return cleaned_output

    def clean_json(data):
        if isinstance(data, dict):
            return {key: clean_json(value) for key, value in data.items()}
        elif isinstance(data, list):
            return [clean_json(item) for item in data]
        elif isinstance(data, str):
            return "" if data.lower() in ["unknown", "na", "null"] else data
        else:
            print(data)
            return data
    
    cleaned_json_data = clean_json(json_data)
    # cleaned_output = json.dumps(cleaned_json_data, ensure_ascii=False)

    return cleaned_json_data

In [ ]:
# import re
# def extract_json(text):
#     if text.startswith("```json"):
#         found = re.findall("```json\n*(.*)\n*```", text)
#         try:
#             result = json.loads(found[0])
#         except IndexError as e:
#             print(e, found, text)
#     else:
#         result = json.loads(text)        
#     return result

In [177]:
for er in three_way_evaluation_results:
    clean_json_output(er['gemini_flash_evaluation'])

In [178]:
three_way_eval_json = [clean_json_output(er['gemini_flash_evaluation']) for er in three_way_evaluation_results]
df_three_way_eval_results = pd.DataFrame(three_way_eval_json)
df_three_way_eval_results.to_csv(output_root_dir + subdir + '/gpt4_llama3_gemma2_evaluation_outcome.csv', index=False)


gpt4_llama3_eval_json = [clean_json_output(er['gemini_flash_evaluation']) for er in gpt4_llama3_evaluation_results]
df_gpt4_llama3_eval_results = pd.DataFrame(gpt4_llama3_eval_json)
df_gpt4_llama3_eval_results.to_csv(output_root_dir + subdir + '/gpt4_llama3_evaluation_outcome.csv', index=False)


gpt4_gemma2_eval_json = [clean_json_output(er['gemini_flash_evaluation']) for er in gpt4_gemma2_evaluation_results]
df_gpt4_gemma2_eval_results = pd.DataFrame(gpt4_gemma2_eval_json)
df_gpt4_gemma2_eval_results.to_csv(output_root_dir + subdir + '/gpt4_gemma2_evaluation_outcome.csv', index=False)

llama3_gemma2_eval_json = [clean_json_output(er['gemini_flash_evaluation']) for er in llama3_gemma2_evaluation_results]
df_llama3_gemma2_eval_results = pd.DataFrame(llama3_gemma2_eval_json)
df_llama3_gemma2_eval_results.to_csv(output_root_dir + subdir + '/llama3_gemma2_evaluation_outcome.csv', index=False)

In [181]:
print(df_three_way_eval_results['winner'].value_counts(),
df_gpt4_llama3_eval_results['winner'].value_counts(),
df_gpt4_gemma2_eval_results['winner'].value_counts(),
df_llama3_gemma2_eval_results['winner'].value_counts())

winner
b    129
c     99
a     76
Name: count, dtype: int64 winner
b    262
a     42
Name: count, dtype: int64 winner
b    269
a     35
Name: count, dtype: int64 winner
b    201
a    103
Name: count, dtype: int64


In [ ]:
df_llama3_gemma2_eval_results['winner'].value_counts()

winner
b    191
a    113
Name: count, dtype: int64